In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-03-03 16:15:53.138409


C:\Users\DELL\AppData\Local\Temp\ipykernel_380\2672588006.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_date = pd.to_datetime(datetime.utcnow())


In [5]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [7]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [9]:
current_date.to_datetime64()

numpy.datetime64('2025-03-03T16:00:00.000000000')

In [11]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [13]:
fetch_data_to

Timestamp('2025-03-03 16:00:00+0000', tz='UTC')

In [15]:
fetch_data_from

Timestamp('2025-02-02 16:00:00+0000', tz='UTC')

In [17]:
from datetime import datetime, timedelta
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date, to_date):
    """
    Fetch historical taxi ride data and shift it forward by 52 weeks.

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Ensure both are timezone-aware
    from_date = from_date.replace(tzinfo=None)  # Remove timezone info
    to_date = to_date.replace(tzinfo=None)

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    print("Historical From Date:", historical_from_date)
    print("Historical To Date:", historical_to_date)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])

    # Ensure pickup_datetime column exists
    if "pickup_datetime" not in rides_from.columns:
        raise ValueError("Error: 'pickup_datetime' column missing from dataset.")

    # Convert pickup_datetime to timezone-naive format
    rides_from["pickup_datetime"] = pd.to_datetime(rides_from["pickup_datetime"]).dt.tz_localize(None)

    # Ensure both timestamps are in the same format (no timezone info)
    historical_from_date = historical_from_date.replace(tzinfo=None)

    # Filter data
    rides_from = rides_from[rides_from["pickup_datetime"] >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to["pickup_datetime"] = pd.to_datetime(rides_to["pickup_datetime"]).dt.tz_localize(None)
        rides_to = rides_to[rides_to["pickup_datetime"] < historical_to_date]
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from

    # Shift the data forward by 52 weeks to simulate recent data
    rides["pickup_datetime"] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=["pickup_location_id", "pickup_datetime"], inplace=True)

    return rides
    

In [19]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

Historical From Date: 2024-02-04 16:00:00
Historical To Date: 2024-03-04 16:00:00
File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [21]:
rides

,pickup_datetime,pickup_location_id
926477,2025-02-12 16:25:44,2
1233178,2025-02-15 16:56:40,2
46632,2025-02-03 09:11:09,3
60767,2025-02-03 12:14:43,3
95189,2025-02-03 18:07:44,3
...,...,...
2919947,2025-03-03 15:56:52,263
2923349,2025-03-03 15:57:24,263
2921202,2025-03-03 15:57:56,263
2922121,2025-03-03 15:59:20,263


In [23]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [25]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-02 16:00:00,2,0
1,2025-02-02 17:00:00,2,0
2,2025-02-02 18:00:00,2,0
3,2025-02-02 19:00:00,2,0
4,2025-02-02 20:00:00,2,0
...,...,...,...
174691,2025-03-03 11:00:00,263,99
174692,2025-03-03 12:00:00,263,91
174693,2025-03-03 13:00:00,263,102
174694,2025-03-03 14:00:00,263,119


In [27]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174696 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174696 non-null  datetime64[ns]
 1   pickup_location_id  174696 non-null  int16         
 2   rides               174696 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [29]:
import hopsworks

api_key = os.getenv('yaeKKye3HP7lw2kL.5sIZ8NQrVMMY6gvZ3IJQXDeSZnaMe81dRVzGE5SLT04EHxNIlV07pio0o74mv6fA')  
project_name = os.getenv('NYC_TAXI_PROJECT_CDA_P1')  

# pip install confluent-kafka
# Initialize connection to Hopsworks  
project = hopsworks.login(  
    api_key_value=api_key,  
    project=project_name  
)  
print(f"Successfully connected to Hopsworks project: {project_name}")


2025-03-03 11:17:02,708 INFO: Initializing external client
2025-03-03 11:17:02,710 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 11:17:04,428 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215640
Successfully connected to Hopsworks project: None


In [31]:
feature_store = project.get_feature_store()

In [33]:
FEATURE_GROUP_NAME = "nyc_time_hourly"
FEATURE_GROUP_VERSION = 1

In [35]:
feature_group = feature_store.get_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
)

In [37]:

# Import necessary modules
from confluent_kafka import Producer  # Ensure this import is correct
import hopsworks
import pandas as pd  # Import Pandas for DataFrame creation

# Initialize Hopsworks project and feature store
project = hopsworks.login()
fs = project.get_feature_store()  # Access the feature store from the project

# Define feature group
feature_group = fs.get_or_create_feature_group(
    name="nyc_time_hourly",
    version=1,
    description="Time-series data at hourly frequency",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour"  # Ensure this is defined as TIMESTAMP in the schema
)

# Define your time-series data as a dictionary
ts_data = {
    "pickup_location_id": [1, 2, 3],  # Example data
    "pickup_hour": ["2023-10-01 12:00", "2023-10-01 13:00", "2023-10-01 14:00"],  # Example data as strings
    "value": [100, 150, 200]  # Example data
}

# Convert the dictionary to a Pandas DataFrame
ts_dataframe = pd.DataFrame(ts_data)

# Convert the `pickup_hour` column to a datetime object
ts_dataframe["pickup_hour"] = pd.to_datetime(ts_dataframe["pickup_hour"])

# Insert data into the feature group
feature_group.insert(ts_dataframe, write_options={"wait_for_job": False})

2025-03-03 11:17:23,316 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 11:17:23,341 INFO: Initializing external client
2025-03-03 11:17:23,343 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 11:17:24,234 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215640


Uploading Dataframe: 100.00% |█████████████████████████████████| Rows 3/3 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: nyc_time_hourly_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215640/jobs/named/nyc_time_hourly_1_offline_fg_materialization/executions


(Job('nyc_time_hourly_1_offline_fg_materialization', 'SPARK'), None)